## REFERENCES

### NOTEBOOKS - classic ML
* [DSR- Gerrit- intro to ML] (http://localhost:8889/notebooks/00_TRAINING/02_DSR/05_ML_Geritt_Week2/Machine%20Learning%20-%20DSR%20Intro.ipynb)
* [DSR-Gerrit- Boosting & XGBoost]
http://localhost:8889/notebooks/00_TRAINING/02_DSR/05_ML_Geritt_Week2/Boosting%20and%20XGBoost.ipynb
* [DSR-Gerrit- RF]
(http://localhost:8889/notebooks/00_TRAINING/02_DSR/05_ML_Geritt_Week2/%20RF%20notebooks/DecisionTrees_Intro.ipynb)
* [DSR-Gerrit-Evaluation models] (http://localhost:8889/notebooks/00_TRAINING/02_DSR/05_ML_Geritt_Week2/exercises/Evaluation%20Solutions.ipynb)
* [DSR-Gerrit- Bayesian Learning]
(http://localhost:8889/notebooks/00_TRAINING/02_DSR/05_ML_Geritt_Week2/exercises/Bayesian%20Learning%20Solutions.ipynb)
* [DSR-Rachel-ML Pipelines]
(http://localhost:8889/notebooks/00_TRAINING/02_DSR/10_DSR_Model_Pipelines_Rachel/1.3%20Pipelines%20.ipynb)
* [DSR-Adam-Python tricks]
(http://localhost:8889/notebooks/00_TRAINING/02_DSR/07_RL_Adam_Week4/dsr_rl/practical/generic_lessons/python_tricks.ipynb)

http://localhost:8889/edit/00_TRAINING/02_DSR/05_ML_Geritt_Week2/ML_Essential_Concepts_Add-Michigan-Notes.txt

### GITHUB repos: 
* **Hands-On Machine Learning w/scikit-learn and TensorFlow book: https://github.com/AMDonati/handson-ml**
* sci-kit learn: https://github.com/AMDonati/scikit-learn/tree/master/sklearn
* https://github.com/AMDonati/data-science-ipython-notebooks
* Data Science w/ Python handbook: https://github.com/AMDonati/PythonDataScienceHandbook

* https://github.com/AMDonati/machine-learning-cheat-sheet

### Specific to time-series: 
* https://github.com/ChadFulton/tsa-notebooks
* https://github.com/maxim5/time-series-machine-learning

## FRAMEWORK
3. Preparing the data for the ML algo:
    * Dimensionality reduction? 
    * Standardization/ normalisation? 
    * split train/val/test
    * do something special for the multi-label classification problem?
4. Select and train the model
5. Model Evaluation: 
    * Fine-tune the model
    * Cross-validtion
    * Metrics plotting

## 0. Imports & functions

In [49]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import sklearn
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
import sklearn.cross_validation as cv
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.metrics import confusion_matrix

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 1. Building a model: template

In [ ]:

mean_regressor=DummyRegressor()
mean.regressor.fit(X,y)
y_pred=mean_regressor.predict(X)

X_train, X_test, y_train, y_test = train_test_split(X, y_setosa, random_state=42)

logreg= LogisticRegression(C=10, penalty="l2",multi_class='multinomial') #penalty= regularization type, here 'l2'
logreg.fit(X_train, y_train)
# C = 1/lambda, the parameter of the sigmoid function
# lambda = 0.001, 0.005, 0.01, 0.05, ..., 1, 5, 10, 50

clf.coef_ # coeff of the linear model
clf.intercept_ # 
print("Train-Score: %.4f, Test-Accuracy: %.4f" % (clf.score(X_train, y_train), clf.score(X_test, y_test)))

from sklearn.metrics import classification_report
report = classification_report(y_test, 
                      logreg.predict(X_test), 
                      #labels=data.feature_names,
                      target_names=data.target_names)
print(report)



RF = RandomForestClassifier(n_estimators=10, max_depth=3)
RF.fit(X_train, y_train)

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
show_decision_function(clf, ax)
ax.set_title('Random Forest')

### Models to train
0. Persistence baseline
1. RF regressor for features importance
2. Logistic Regression for times-series forecast
2. RF/ XGBoost for time-series prediction
3. Linear model with sample weighted loss

### About the use of RF as a features importance selector: 
* https://towardsdatascience.com/running-random-forests-inspect-the-feature-importances-with-this-code-2b00dd72b92e
* https://medium.com/the-artificial-impostor/feature-importance-measures-for-tree-models-part-i-47f187c1a2c3
* https://chrisalbon.com/machine_learning/trees_and_forests/feature_selection_using_random_forest/

### Run a random forest on the 2 possible labels (NHY & MODEADL)
### Take different targets values to see how it evolves. 
### Take also all samples with the mean. 
* DiagFeat
* Non-Motor Assessments
* Motor Assessments
* General Medical Data

# I. Features importance & Non TS classification of Disease Progression

## 1. Data preparation

In [2]:
file_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/output/pre-processing/dfFinal.csv'
df=pd.read_csv(file_path)
df.drop(columns='Unnamed: 0',inplace=True)
df.drop(columns=['EVENT_ID','INFODT'],inplace=True)
#df.set_index(keys=['PATNO'],inplace=True)
df.head()

,PATNO,INFODT_date,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,NP2SPCH,NP2SALV,...,DXPOSINS,DXOTHSX,DOMSIDE,num_visits,VISIT_ID,visitsdiff_days,lastDate_diff_days,PDDXDT_diff_days,PDMEDT_diff_days,PDSURGDT_diff_days
0,3001,2011-02-01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,2,7,1,0,2707,306,0,0
1,3001,2012-03-01,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,2,7,2,394,2313,700,0,0
2,3001,2013-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,2,7,3,426,1887,1126,0,0
3,3001,2014-04-01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,2,7,4,335,1552,1461,0,0
4,3001,2015-04-01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,2,7,5,365,1187,1826,0,0


In [39]:
dfAgg=df.groupby('PATNO').agg('median')
dfAgg.drop(columns=['visitsdiff_days','VISIT_ID'],inplace=True)
dfAgg['NHY']=dfAgg['NHY'].apply(lambda x: round(x,0))
dfAgg.head()

,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,NP2SPCH,NP2SALV,NP2SWAL,NP2EAT,...,DXRIGID,DXBRADY,DXPOSINS,DXOTHSX,DOMSIDE,num_visits,lastDate_diff_days,PDDXDT_diff_days,PDMEDT_diff_days,PDSURGDT_diff_days
PATNO,,,,,,,,,,,,,,,,,,,,,
3001,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,2.0,7.0,1552.0,1461.0,0.0,0.0
3002,1.0,0.0,1.0,1.0,0.0,0.0,1.5,2.5,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,6.0,1567.5,1504.5,0.0,0.0
3003,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,7.0,1552.0,1857.0,0.0,0.0
3006,0.0,0.0,0.5,1.0,0.0,0.0,2.5,0.0,1.0,1.0,...,1.0,1.0,0.0,0.0,2.0,2.0,2465.5,333.5,0.0,0.0
3007,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,1.0,1.0,0.0,1.0,2.0,1.0,2648.0,90.0,0.0,0.0


In [53]:
dfAgg['NHY'].value_counts()

2.0    692
0.0    683
1.0    203
3.0     69
4.0     24
5.0      3
Name: NHY, dtype: int64

### Take care of the classes unbalance

## 2. Split train, test

In [41]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dfAgg, test_size=0.2, random_state=0)
train.to_pickle("./train_nonTS.pickle")
test.to_pickle("./test_nonTS.pickle")
train.head()

,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,NP2SPCH,NP2SALV,NP2SWAL,NP2EAT,...,DXRIGID,DXBRADY,DXPOSINS,DXOTHSX,DOMSIDE,num_visits,lastDate_diff_days,PDDXDT_diff_days,PDMEDT_diff_days,PDSURGDT_diff_days
PATNO,,,,,,,,,,,,,,,,,,,,,
3608,1.0,1.0,1.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,2.0,2282.5,1034.5,0.0,0.0
3909,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,2068.0,31.0,0.0,0.0
4003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,2557.0,607.0,0.0,0.0
52062,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,3.0,577.0,1065.0,0.0,0.0
40240,0.0,0.0,0.0,0.5,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,2.0,2.0,1261.5,3516.5,0.0,0.0


In [42]:
X_train = train.loc[:,train.columns!='NHY']
y_train = train['NHY']
X_test = test.loc[:,train.columns!='NHY']
y_test = test[('NHY')]
X_train.shape

(1339, 371)

## 3. Model Training

In [43]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(multi_class='multinomial',solver='newton-cg')
logreg.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

## 4. Evaluation: Calculating, plotting metrics

In [46]:
print(logreg.coef_,logreg.intercept_)

[[-5.83376589e-01  1.26562966e-01 -1.79959057e-02 ... -1.88268692e-03
  -1.52819988e-03 -2.05342073e-02]
 [ 7.76246533e-02  8.96258755e-01  8.79021986e-03 ... -5.64950208e-04
  -5.73468342e-03 -2.18241867e-02]
 [ 2.72009771e-01 -5.96940611e-01 -1.82490408e-01 ...  4.58999370e-05
   8.12706928e-03  1.44082846e-02]
 [ 3.48077793e-01 -3.08449945e-01  1.68132751e-01 ... -1.01502279e-05
  -8.31753393e-04  1.42231105e-02]
 [-1.35095664e-01 -1.25792968e-01  8.11691866e-02 ...  9.86961049e-04
  -1.93117227e-05  1.43767060e-02]
 [ 2.07600353e-02  8.36180358e-03 -5.76058433e-02 ...  1.42492637e-03
  -1.31208679e-05 -6.49707138e-04]] [-1.76103438e-03  9.36517018e-03 -8.23851286e-03 -4.82671241e-05
  1.36399686e-04  5.46244495e-04]


In [48]:
print("Train-Score: %.4f, Test-Accuracy: %.4f" % (logreg.score(X_train, y_train), logreg.score(X_test, y_test)))

Train-Score: 0.9866, Test-Accuracy: 0.8448


**model overfitting**: to retrain with parameters tuning.  
**Deal with unbalance classes here**

### Confusion Matrix

In [51]:
y_pred=logreg.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cm

array([[132,   1,   0,   0,   0,   0],
       [  0,  15,  19,   1,   0,   0],
       [  2,  14, 130,   2,   0,   0],
       [  0,   0,   8,   3,   2,   1],
       [  0,   0,   0,   2,   3,   0],
       [  0,   0,   0,   0,   0,   0]])

### Decision function plot

## 5. Cross-Validation